In [2]:
import pandas as pd

input = 'input.txt'
input_df = pd.read_csv(input, sep='\t', dtype={'start': int, 'gene': str, 'PSI': float})
input_df

,gene,start,end,PSI
0,MET,116771846,116771847,0.8


In [21]:
import requests
import pandas as pd
import re

# TODO: need to double check this map
tissue_map = {

    'ACC': ['Adrenal gland'],
    'BLCA': ['Bladder'],
    'BRCA': ['Breast'],
    'CESC': ['Cervix Uteri'],
    'COAD': ['Colon'],
    'ESCA': ['Esophagus'],
    'HNSC': ['Salivary gland', 'Esophagus'],
    'KIRP': ['Kidney'],
    'KIRC': ['Kidney'],
    'KICH': ['Kidney'],
    'LIHC': ['Liver'],
    'LUSC': ['Lung'],
    'LUAD': ['Lung'],
    'OV': ['Ovary'],
    'PAAD': ['Pancreas'],
    'PRAD': ['Prostate'],
    'READ': ['Colon'],
    'SKCM': ['Skin'],
    'STAD': ['Stomach'],
    'TGCT': ['Testis'],
    'THCA': ['Thyroid'],
    'UCEC': ['Uterus'],
    'UCS': ['Uterus'],
    'UVM': ['Eye'],
    'THYM': ['Thymus'],

    # need to double check these
    'CHOL': ['Liver'],
    'DLBC': ['Blood', 'Spleen'],
    'GBM': ['Brain'],
    'LGG': ['Brain'],
    'LAML': ['Blood'],
    'MESO': ['Lung'],
    'PCPG': ['Adrenal gland', 'Nerve'],
    'SARC': ['Muscle', 'Adipose Tissue', 'Nerve'],
}

# Add psi_diff and significance columns
def compute_psi_diff(tcga_str, gtex_str):
    tcga_psi = parse_psi(tcga_str, scale_by=100)
    gtex_psi = parse_psi(gtex_str, scale_by=100)

    psi_diff_entries = []
    significance_entries = []

    for cancer, tcga_val in tcga_psi.items():
        gtex_tissues = tissue_map.get(cancer, [])
        normal_vals = [gtex_psi[t] for t in gtex_tissues if t in gtex_psi]
        if normal_vals:
            normal_avg = sum(normal_vals) / len(normal_vals)
            diff = tcga_val - normal_avg
            psi_diff_entries.append(f"{cancer}:{diff:.2f}")

            if abs(diff) >= 0.2:
                significance_entries.append(f"{cancer}:strong_candidate")
            elif abs(diff) >= 0.1:
                significance_entries.append(f"{cancer}:potential_candidate")

    psi_diff_str = ",".join(psi_diff_entries)
    significance_str = ",".join(significance_entries)

    return pd.Series([psi_diff_str, significance_str])



def parse_ctat_file(ctat_filepath):
    ctat_df = pd.read_csv(ctat_filepath, sep='\t', dtype=str)

    intron_split = ctat_df['intron'].str.extract(r'chr(\w+):(\d+)-(\d+)')
    ctat_df['chromosome'] = intron_split[0]
    ctat_df['intron_start'] = intron_split[1].astype(int)
    ctat_df['intron_end'] = intron_split[2].astype(int)

    # Split 'genes' into multiple rows if it contains multiple genes, and deduplicate
    def split_and_dedup(gene_str):
        if pd.isna(gene_str):
            return []
        parts = re.split(r',|--', gene_str)
        return list(dict.fromkeys([p.strip() for p in parts if p.strip()]))

    ctat_df['gene_entries'] = ctat_df['genes'].apply(split_and_dedup)
    ctat_df = ctat_df.explode('gene_entries', ignore_index=True)

    # Further split each gene entry into gene_name and ensg_id using ^
    genes_split = ctat_df['gene_entries'].str.split('^', n=1, expand=True)
    ctat_df['gene_name'] = genes_split[0]
    ctat_df['ensg_ids'] = genes_split[1]
    ctat_df[['psi_diff', 'significance']] = ctat_df.apply(
        lambda row: compute_psi_diff(row['TCGA_sample_counts'], row['GTEx_sample_counts']),
        axis=1
    )

    return ctat_df[['chromosome', 'intron_start', 'intron_end', 'gene_name', 'ensg_ids',
                    'TCGA_sample_counts', 'GTEx_sample_counts', 'variant_name', 'psi_diff', 'significance']]


# TODO need transcript id
def fetch_exon_number(ensg_ids, intron_start, intron_end):
    # server = "https://rest.ensembl.org"
    # ensg_id_list = ensg_ids.split('^')
    # exon_numbers = []

    # for ensg_id in ensg_id_list:
    #     ensg_id = ensg_id.split('.')[0]  # Remove version number if present
    #     ext = f"/lookup/id/{ensg_id}?expand=1"
    #     headers = {"Content-Type": "application/json"}
    #     response = requests.get(server+ext, headers=headers)

    #     if not response.ok:
    #         continue

    #     data = response.json()
    #     found_match = False
    #     if 'Transcript' in data:
    #         for transcript in data['Transcript']:
    #             for exon in transcript.get('Exon', []):
    #                 exon_start = exon['start']
    #                 exon_end = exon['end']
    #                 if intron_start <= exon_start and intron_end >= exon_end:
    #                     exon_numbers.append(exon['exon_number'])
    #                     found_match = True
    #                     break
    #             if found_match:
    #                 break
    #     if not found_match:
    #         return ""

    # return exon_numbers[0] if exon_numbers else ""
    return ""

def parse_psi(sample_counts, scale_by=1):
    entries = str(sample_counts).split(',')
    result = {}
    for entry in entries:
        parts = entry.split(':')
        if len(parts) == 3:
            result[parts[0]] = float(parts[2]) / scale_by
    return result

ctat_file = 'cancer_introns.GRCh38.Jun232020.tsv'
ctat_file_test = 'cancer_introns.GRCh38.Jun232020_test.tsv'

# Parse CTAT file
ctat_df = parse_ctat_file(ctat_file)
ctat_df

,chromosome,intron_start,intron_end,gene_name,ensg_ids,TCGA_sample_counts,GTEx_sample_counts,variant_name,psi_diff,significance
0,5,104080125,104080907,RP11-138J23.1,ENSG00000251026.1,"ESCA:97:50.79,READ:44:42.31,COAD:120:39.22,LUS...",NaN,NaN,,
1,6,30260085,30260332,HLA-L,ENSG00000243753.4,"KIRC:180:30.00,DLBC:11:22.92,BRCA:209:17.46,ME...",NaN,NaN,,
2,17,68251179,68256813,AMZ2,ENSG00000196704.10,"THYM:119:97.54,LGG:503:96.55,PCPG:178:96.22,UV...",NaN,NaN,,
3,3,178749237,178841544,KCNMB2-AS1,ENSG00000237978.4,"LUSC:354:65.68,OV:167:63.98,ESCA:94:49.21,UCS:...",NaN,NaN,,
4,3,178749237,178859488,KCNMB2-AS1,ENSG00000237978.4,"OV:174:66.67,ESCA:121:63.35,LUSC:298:55.29,UCE...",NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...
25855,2,178320403,178324176,OSBPL6,ENSG00000079156.15,"LUAD:5:0.89,STAD:2:0.50,LUSC:2:0.37",Lung:2:0.47,NaN,"LUAD:0.00,LUSC:-0.00",
25856,7,55109959,55155829,EGFR,ENSG00000146648.14,GBM:2:1.18,NaN,EGFRvIIIb,,
25857,7,55200414,55202516,EGFR,ENSG00000146648.14,"GBM:1:0.59,LGG:1:0.19",NaN,EGFRvIVb,,
25858,7,55161632,55170306,EGFR,ENSG00000146648.14,NaN,NaN,EGFRvIIb,,


In [22]:
# --------------------- Parse GTF and extract exon info --------------------- #
import pandas as pd
import gzip
import re

def parse_gtf(gtf_path):
    rows = []

    open_func = gzip.open if gtf_path.endswith('.gz') else open
    with open_func(gtf_path, 'rt') as f:
        for line in f:
            if line.startswith('#'):
                continue
            fields = line.strip().split('\t')
            if fields[2] != 'exon':
                continue

            chrom, source, feature_type, start, end, score, strand, frame, attributes = fields

            # Parse attributes
            attr_dict = dict(re.findall(r'(\S+) "([^"]+)"', attributes))
            gene_id = attr_dict.get('gene_id')
            gene_version = attr_dict.get('gene_version')
            transcript_id = attr_dict.get('transcript_id')
            transcript_version = attr_dict.get('transcript_version')
            gene_name = attr_dict.get('gene_name', '')
            exon_number = attr_dict.get('exon_number', '')

            # Combine ID with version for gene and transcript
            # use full gene id or versioned id
            # full_gene_id = f"{gene_id}.{gene_version}" if gene_version else gene_id
            full_gene_id = gene_id
            # if not using full gene id, then don't use versioned transcript id
            # full_transcript_id = f"{transcript_id}.{transcript_version}" if transcript_version else transcript_id
            full_transcript_id = transcript_id

            rows.append({
                'gene_name': gene_name,
                'gene_id': full_gene_id,
                'transcript_id': full_transcript_id,
                'exon_number': exon_number,
                'exon_start': int(start),
                'exon_end': int(end)
            })

    return pd.DataFrame(rows)

# Example usage
gtf_file = 'Homo_sapiens.GRCh38.111.gtf.gz'
gtf_df = parse_gtf(gtf_file)
gtf_df
# View or save
# print(gtf_df.head())
# gtf_df.to_csv("exon_table.csv", index=False)

,gene_name,gene_id,transcript_id,exon_number,exon_start,exon_end
0,DDX11L17,ENSG00000279928,ENST00000624431,1,182696,182746
1,DDX11L17,ENSG00000279928,ENST00000624431,2,183132,183216
2,DDX11L17,ENSG00000279928,ENST00000624431,3,183494,183571
3,DDX11L17,ENSG00000279928,ENST00000624431,4,183740,183901
4,DDX11L17,ENSG00000279928,ENST00000624431,5,183981,184174
...,...,...,...,...,...,...
1650900,,ENSG00000271254,ENST00000612640,16,6101,6370
1650901,U1,ENSG00000275987,ENST00000618083,1,30437,30580
1650902,,ENSG00000268674,ENST00000601199,1,35407,35916
1650903,,ENSG00000277475,ENST00000612315,1,31698,32528


In [ ]:
# --------------------- Annotate CTAT --------------------- #
def annotate_ctat_with_exon(ctat_df, gtf_df):
    ctat_df['intron_start'] = ctat_df['intron_start'].astype(int)
    ctat_df['intron_end'] = ctat_df['intron_end'].astype(int)

    ctat_df['transcript_id'] = ''
    ctat_df['exon_number'] = ''
    ctat_df['exon_start'] = ''
    ctat_df['exon_end'] = ''

    # use unversioned ensg id
    gtf_grouped = gtf_df.groupby('gene_id')

    for idx, row in ctat_df.iterrows():
        ensg_id = row['ensg_ids'].split('.')[0]
        intron_start = row['intron_start']
        intron_end = row['intron_end']

        if ensg_id not in gtf_grouped.groups:
            continue

        exons = gtf_grouped.get_group(ensg_id)

        match = exons[(exons['exon_start'] > intron_start) & (exons['exon_end'] < intron_end)]
        if not match.empty:
            ctat_df.at[idx, 'transcript_id'] = ','.join(match['transcript_id'].astype(str))
            ctat_df.at[idx, 'exon_number'] = ','.join(match['exon_number'].astype(str))
            ctat_df.at[idx, 'exon_start'] = ','.join(match['exon_start'].astype(str))
            ctat_df.at[idx, 'exon_end'] = ','.join(match['exon_end'].astype(str))

    return ctat_df[['chromosome', 'intron_start', 'intron_end', 'gene_name', 'ensg_ids','transcript_id', 'exon_number', 'exon_start', 'exon_end',
                    'TCGA_sample_counts', 'GTEx_sample_counts', 'variant_name', 'psi_diff', 'significance'
                    ]]

# Annotate CTAT with exon information
annotated_ctat_df = annotate_ctat_with_exon(ctat_df, gtf_df)
annotated_ctat_df


# quesition: mapping by versioned ensg id or unversioned ensg id?

,chromosome,intron_start,intron_end,gene_name,ensg_ids,transcript_id,exon_number,exon_start,exon_end,TCGA_sample_counts,GTEx_sample_counts,variant_name,psi_diff,significance
0,5,104080125,104080907,RP11-138J23.1,ENSG00000251026.1,,,,,"ESCA:97:50.79,READ:44:42.31,COAD:120:39.22,LUS...",NaN,NaN,,
1,6,30260085,30260332,HLA-L,ENSG00000243753.4,,,,,"KIRC:180:30.00,DLBC:11:22.92,BRCA:209:17.46,ME...",NaN,NaN,,
2,17,68251179,68256813,AMZ2,ENSG00000196704.10,"ENST00000674770,ENST00000674770,ENST0000061229...","6,7,6,7,6,7,6,7,5,6,4,5,6,6,6,6,6,7,4,5,4,5,3,4,1","68254404,68255700,68254404,68255700,68254404,6...","68254567,68255876,68254567,68255876,68254567,6...","THYM:119:97.54,LGG:503:96.55,PCPG:178:96.22,UV...",NaN,NaN,,
3,3,178749237,178841544,KCNMB2-AS1,ENSG00000237978.4,"ENST00000437488,ENST00000451742,ENST0000045174...","3,4,5,1","178757080,178801793,178757080,178801793","178757133,178801934,178757136,178801936","LUSC:354:65.68,OV:167:63.98,ESCA:94:49.21,UCS:...",NaN,NaN,,
4,3,178749237,178859488,KCNMB2-AS1,ENSG00000237978.4,"ENST00000437488,ENST00000432385,ENST0000045174...","3,2,3,4,5,2,1","178757080,178841545,178841545,178801793,178757...","178757133,178841616,178841616,178801934,178757...","OV:174:66.67,ESCA:121:63.35,LUSC:298:55.29,UCE...",NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25855,2,178320403,178324176,OSBPL6,ENSG00000079156.15,ENST00000477097,1,178323570,178323780,"LUAD:5:0.89,STAD:2:0.50,LUSC:2:0.37",Lung:2:0.47,NaN,"LUAD:0.00,LUSC:-0.00",
25856,7,55109959,55155829,EGFR,ENSG00000146648.14,"ENST00000344576,ENST00000344576,ENST0000034457...","2,3,4,5,6,7,2,3,4,5,6,7,2,3,4,5,6,2,3,4,5,6,7,...","55142286,55143305,55146606,55151294,55152546,5...","55142437,55143488,55146740,55151362,55152664,5...",GBM:2:1.18,NaN,EGFRvIIIb,,
25857,7,55200414,55202516,EGFR,ENSG00000146648.14,"ENST00000275493,ENST00000275493,ENST0000045508...","25,26,24,25,25,26,3,4,1,2","55201188,55201735,55201188,55201735,55201188,5...","55201355,55201782,55201355,55201782,55201355,5...","GBM:1:0.59,LGG:1:0.19",NaN,EGFRvIVb,,
25858,7,55161632,55170306,EGFR,ENSG00000146648.14,"ENST00000344576,ENST00000344576,ENST0000027549...","14,15,14,15,13,14,14,15,16,14,15,1,2","55163733,55165280,55163733,55165280,55163733,5...","55163823,55165437,55163823,55165437,55163823,5...",NaN,NaN,EGFRvIIb,,


In [ ]:
annotated_ctat_df.to_csv('CTAT_Splicing_enriched.tsv', sep='\t', index=False)


In [5]:

# match and extract info
def match_and_extract_info(input_df, ctat_df):
    matched_rows = []

    for _, row in input_df.iterrows():
        gene = row['gene']
        start_pos = row['start']
        end_pos = row['end']
        psi_value = row['PSI']

        matched_ctat = ctat_df[(ctat_df['gene_name'] == gene) &
                               (ctat_df['intron_start'] <= start_pos) &
                               (ctat_df['intron_end'] >= start_pos)]

        for _, matched in matched_ctat.iterrows():
            matched_rows.append({
                'gene': gene,
                'start': start_pos,
                'end': end_pos,
                'PSI': psi_value,
                'exon_number': matched['exon_number'],
                'TCGA_sample_counts': matched['TCGA_sample_counts'],
                'GTEx_sample_counts': matched['GTEx_sample_counts'],
                'variant_name': matched['variant_name']
            })

    return pd.DataFrame(matched_rows)


result_df = match_and_extract_info(input_df, ctat_df)
result_df.head()


,gene,start,end,PSI,exon_number,TCGA_sample_counts,GTEx_sample_counts,variant_name
0,MET,116771846,116771847,0.8,,"LUAD:12:2.14,LUSC:2:0.37,LGG:1:0.19",NaN,METx14del
